In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup

import pandas as pd 

import os, time, re, datetime, random




In [2]:
# 本机命令行指定端口运行浏览器 
# cd C:\Program Files (x86)\Google\Chrome\Application
os.chdir(r'C:\Program Files (x86)\Google\Chrome\Application')
# chrome.exe -remote-debugging-port=9222
cmd = 'chrome.exe -remote-debugging-port=9222'
os.popen(cmd)

# 调用已经打开的浏览器  
s = Service(r"C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe")
options = webdriver.ChromeOptions()
options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
driver = webdriver.Chrome(service=s,options=options)
driver.get('https://hotel.fliggy.com/hotel_list3.htm?null&_output_charset=utf8')
# 弹出页面后，看是否登录，如果没有，60秒内登录 验证 
time.sleep(6)
print(driver.title) 

北京酒店,北京酒店预订,北京宾馆住宿推荐-飞猪酒店


In [3]:
def get_range_date():
    today = datetime.date.today()  # 获取当前日期  datetime.date(2022, 8, 12)
    oneday = datetime.timedelta(days=1)  
    eightday = datetime.timedelta(days=7)
    end_date = today - oneday  
    start_date = today - eightday
    return start_date, end_date

s_date, e_date = get_range_date()

In [4]:
def get_sub_shop_info(driver):
    # 酒店名 
    hotel_name = driver.find_element(By.XPATH, '//div[@class="info"]/div[1]/h2').text
    # fz_score = driver.find_element(By.XPATH, '//div[@class="other"]/ul/li[@class="rate"]/a').text  # 这个位置可能没有评分 
    fz_score = driver.find_element(By.XPATH, '//li[@class="taRate"]/div[@class="rate"]').text
    # print(hotel_name, fz_score)
    driver.execute_script('window.scrollBy(0,%d)'%(random.randint(600, 1000)))
    # TODO 判断评论是否翻页
    # 获取当前页所有评论的父级元素
    try:
        comms_driver_lst = driver.find_elements(By.XPATH, '//ul[@class="review-list"]/li') 
    except:
        return pd.DataFrame([{'酒店名': hotel_name, '飞猪评分': fz_score, '评论数量': 0, '差评数量': 0, '差评内容': '', '差评时间': '', '差评分数': '', '商家是否回复': ''}])
    coms_lst = []
    for i in range(len(comms_driver_lst)):
        # 判断评论时间是否在时间段内
        com_time_str = comms_driver_lst[i].find_element(By.XPATH, 'div[@class="tb-r-bd"]/div/div[@class="tb-r-info"]/span').text
        com_time_str = com_time_str.replace('[', '').replace(']', '')
        com_time = datetime.datetime.strptime(com_time_str, '%Y-%m-%d %H:%M')
        if com_time.date() > e_date:
            continue 
        if com_time.date() < s_date:
            break 
        dct = {}
        # 取内容、分数、时间 商家是否回复 
        com_content_par = comms_driver_lst[i].find_elements(By.XPATH, 'div[@class="tb-r-bd"]/div/div[@class="tb-r-cnt"]')
        com_content = com_content_par[0].text
        score_origin = comms_driver_lst[i].find_element(By.XPATH, 'div[@class="tb-r-bd"]/div/div[@class="groupIcon"]/div').get_attribute('style')
        com_score = int(re.findall("\d+", score_origin)[0]) / 20 
        reply_label = '是' if len(com_content_par) > 1 else '否'
        
        dct['评论内容'] = com_content
        dct['评论时间'] = com_time.strftime('%Y-%m-%d %H:%M')
        dct['评论分数'] = com_score
        dct['商家是否回复'] = reply_label
        coms_lst.append(dct) 
    df_shop_sub = pd.DataFrame([[hotel_name, fz_score]], columns=['酒店名', '飞猪评分']) 
    df_com_sub = pd.DataFrame(coms_lst, columns=['评论内容', '评论时间', '评论分数', '商家是否回复'])
    df_shop_sub['评论数量'] = df_com_sub['评论时间'].count() 
    df_shop_sub['差评数量'] = df_com_sub[df_com_sub['评论分数']<=3]['评论时间'].count() 
    df_negative = df_com_sub[df_com_sub['评论分数']<=3]
    df_negative.columns = ['差评内容', '差评时间', '差评分数', '商家是否回复']
    df_negative['酒店名'] = hotel_name
    df_sub = pd.merge(df_shop_sub, df_negative, 'left', '酒店名')
    # '酒店名', '飞猪评分', '评论数量', '差评数量', '差评内容', '差评时间', '差评分数', '商家是否回复'
    return df_sub


In [ ]:
# 目标：'门店名', '携程评分', '评价数量', '差评数量', '差评内容', '差评分数', '商家是否回复', '差评时间' 
# 网鱼电酒分布城市：上海市 杭州市 广州市 深圳市 武汉市 苏州市 南京市 长沙市 
# 先输入 "网鱼电竞酒店" 然后输入城市名 回车 
driver = webdriver.Chrome(service=s,options=options)
ActionChains(driver).move_by_offset(200, 100).click().perform() 
time.sleep(5)
input_hotel_vague = driver.find_element(By.XPATH, '//div[@class="search-field pr10"]/div/input')
input_hotel_vague.send_keys(Keys.CONTROL, 'a')
input_hotel_vague.send_keys('网鱼电竞酒店') 
time.sleep(1) 
ActionChains(driver).move_by_offset(200, 100).click().perform()  # 鼠标左键点击， 200为x坐标， 100为y坐标  .context_click() 鼠标右键
target_city = ['上海', '杭州', '广州', '深圳', '武汉', '苏州', '南京', '长沙'] 
df_result = pd.DataFrame([], columns=['酒店名', '飞猪评分', '评论数量', '差评数量', '差评内容', '差评时间', '差评分数', '商家是否回复'])
for city in target_city:
    input_city = driver.find_element(By.XPATH, '//div[@class="search-field pr30"]/div/input')
    input_city.send_keys(Keys.CONTROL, 'a') 
    input_city.send_keys('%s'%city) 
    time.sleep(1)
    # input_city.send_keys(Keys.ENTER)
    driver.find_element(By.XPATH, '//div[@class="search-submitarea"]/button').click()
    time.sleep(3) 
    driver.execute_script('window.scrollBy(0,%d)'%(random.randint(500, 800)))
    # 依次点击页面中的酒店 TODO 需要新增判断是否翻页 暂时不需要，因为没有一个城市网鱼电酒数量超过20 
    # 获取总的搜索结果数  
    # hotel_result_cnt = driver.find_element(By.XPATH, '//div[@class="head-result"]/span').text 
    
    df_result_city = pd.DataFrame([], columns=['酒店名', '飞猪评分', '评论数量', '差评数量', '差评内容', '差评时间', '差评分数', '商家是否回复'])
    # 当前页面酒店连接集合
    # cur_hotel_lst = driver.find_elements(By.XPATH, '//h5[@class="row-title"]/a')
    cur_hotel_lst = driver.find_elements(By.XPATH, '//div[@id="J_List"]/div[@class="list-row J_ListRow J_LazyZoom"]')
    # 当前页面酒店评分集合 
    # cur_hotel_score_lst = driver.find_elements(By.XPATH, '//div[@class="row-sub-right fright"]/a/p[@class="score"]/span[@class="value"]') 
    # 点击某个页面 获取酒店名 评分 评论 
    for i in range(len(cur_hotel_lst)):
        hotel = cur_hotel_lst[i].find_element(By.XPATH, 'div/div[@class="row-center"]/div/h5/a')
        # print(hotel.text)
        if '网鱼电竞酒店' not in hotel.text:
            break 
        # 点击跳转到酒店详情页 TODO 会拒绝访问，目前行不通
        # hotel.click()
        # time.sleep(random.randint(4, 6))
        # windows = driver.window_handles
        # driver.switch_to.window(windows[-1]) 
        # # print(driver.title)
        # df_sub = get_sub_shop_info(driver) 
        # df_result_city = pd.concat([df_result_city, df_sub])
        # time.sleep(2)
        # driver.close() 
        # driver.switch_to.window(windows[0]) 
        # time.sleep(random.randint(2,4)) 

        # 在酒店列表页保存酒店名和评分 
        dct = {}
        dct['酒店名'] = hotel.text 
        try: 
            hotel_score = cur_hotel_lst[i].find_element(By.XPATH, 'div/div[@class="row-sub-right fright"]/a/p[@class="score"]/span[@class="value"]').text
        except:
            hotel_score = '暂无评分'
        dct['飞猪评分'] = hotel_score
        dct['评论数量'] = 0
        dct['差评数量'] = 0
        dct['差评内容'] = ''
        dct['差评时间'] = '' 
        dct['差评分数'] = ''
        dct['商家是否回复'] = ''
        df_sub = pd.DataFrame([dct])
        df_result_city = pd.concat([df_result_city, df_sub])
        
    df_result = pd.concat([df_result, df_result_city])
df_result.to_excel('D:/Document/local_rps/python3/data/%s_飞猪评分和近7天评论.xlsx'%str(datetime.date.today()))

